## UNData API Exercise

In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.

You'll make use of the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation). Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the [Basic Call Structure article](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581).

In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

1. Use the API to get all available data for the _GDP per capita, PPP (constant 2017 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [2]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?format=json'
response= requests.get(endpoint)

In [3]:
response

<Response [200]>

In [4]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD'

params = {
    'format' : 'json',
    'Per_page': '20000',
}

response = requests.get(endpoint, params=params)

response

<Response [200]>

In [5]:
res = response.json()
len(res[1])

17290

In [6]:
un_df_1 = pd.json_normalize(res[1])
un_df_1 = un_df_1.rename(columns={"value" : "GDP"})
un_df_1.head(2)

,countryiso3code,date,GDP,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,3968.963751,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
1,AFE,2023,3948.142721,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern


2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [7]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN'

params = {
    'format' : 'json',
    'Per_page': '20000',
}

response = requests.get(endpoint, params=params)

response

<Response [200]>

In [8]:
res = response.json()
len(res[1])

17290

In [9]:
un_df_2 = pd.json_normalize(res[1])
un_df_2 = un_df_2.rename(columns={"value" : "Life_Expectancy"})
un_df_2.head(2)

,countryiso3code,date,Life_Expectancy,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,NaN,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
1,AFE,2023,65.146291,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern


3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [10]:
un_merged = pd.merge(un_df_1, un_df_2, on=['date', 'country.value'], how = "inner" )
#"on" selecting the filter columns to make sure we matched during the merge

In [11]:
un_merged_rough = un_merged[["GDP", "date", "country.value", "Life_Expectancy"]]
#filtered to columns i wanted

In [12]:
un_merged_clean = un_merged_rough.rename(columns={"country.value" : "Country", "date" : "Year" })
un_merged_clean.head(2)
#renamed columns

,GDP,Year,Country,Life_Expectancy
0,3968.963751,2024,Africa Eastern and Southern,NaN
1,3948.142721,2023,Africa Eastern and Southern,65.146291


4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [13]:
endpoint = 'https://api.worldbank.org/v2/country/'
params = {
    'format' : 'json',
    'Per_page': '20000',
}

response = requests.get(endpoint, params=params)
response

<Response [200]>

In [14]:
res = response.json()
len(res[1])

296

In [15]:
country_region = pd.json_normalize(res[1])
country_region_clean = country_region[country_region['region.id'] != "NA"]

In [16]:
country_region_clean['region.value'].unique()

array(['Latin America & Caribbean ',
       'Middle East, North Africa, Afghanistan & Pakistan',
       'Sub-Saharan Africa ', 'Europe & Central Asia',
       'East Asia & Pacific', 'South Asia', 'North America'], dtype=object)

In [17]:
country_region_clean.head(2)

,id,iso2Code,name,capitalCity,longitude,latitude,region.id,region.iso2code,region.value,adminregion.id,adminregion.iso2code,adminregion.value,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,LCN,ZJ,Latin America & Caribbean,,,,HIC,XD,High income,LNX,XX,Not classified
2,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,MEA,ZQ,"Middle East, North Africa, Afghanistan & Pakistan",MNA,XQ,"Middle East, North Africa, Afghanistan & Pakis...",LIC,XM,Low income,IDX,XI,IDA


In [18]:
country_region_clean = country_region_clean.rename( columns = {'name':'Country'})
cr_for_merge = country_region_clean[['Country', 'capitalCity', 'region.value', 'incomeLevel.value']]

In [19]:
gdp_le_region = pd.merge(un_merged_clean, cr_for_merge, on = "Country", how= "inner")
gdp_le_region = gdp_le_region.rename (columns= {'region.value': 'Region', 'incomeLevel.value':"Income Level", 'capitalCity': "Capital"})
gdp_le_region

,GDP,Year,Country,Life_Expectancy,Capital,Region,Income Level
0,NaN,2024,Afghanistan,NaN,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
1,1983.812620,2023,Afghanistan,66.035,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
2,1981.710168,2022,Afghanistan,65.617,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
3,2144.166570,2021,Afghanistan,60.417,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
4,2769.685745,2020,Afghanistan,61.454,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
...,...,...,...,...,...,...,...
14100,NaN,1964,Zimbabwe,55.431,Harare,Sub-Saharan Africa,Lower middle income
14101,NaN,1963,Zimbabwe,54.942,Harare,Sub-Saharan Africa,Lower middle income
14102,NaN,1962,Zimbabwe,54.453,Harare,Sub-Saharan Africa,Lower middle income
14103,NaN,1961,Zimbabwe,53.966,Harare,Sub-Saharan Africa,Lower middle income


In [20]:
#dropped NAN values in Life expectancy and gdp
gdp_le_region_final = gdp_le_region.dropna(subset = ['Life_Expectancy', 'GDP'])
gdp_le_region_final

,GDP,Year,Country,Life_Expectancy,Capital,Region,Income Level
1,1983.812620,2023,Afghanistan,66.035,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
2,1981.710168,2022,Afghanistan,65.617,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
3,2144.166570,2021,Afghanistan,60.417,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
4,2769.685745,2020,Afghanistan,61.454,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
5,2927.245144,2019,Afghanistan,62.941,Kabul,"Middle East, North Africa, Afghanistan & Pakistan",Low income
...,...,...,...,...,...,...,...
14070,3965.727862,1994,Zimbabwe,52.537,Harare,Sub-Saharan Africa,Lower middle income
14071,3634.747632,1993,Zimbabwe,53.976,Harare,Sub-Saharan Africa,Lower middle income
14072,3649.889072,1992,Zimbabwe,55.602,Harare,Sub-Saharan Africa,Lower middle income
14073,4126.401997,1991,Zimbabwe,57.037,Harare,Sub-Saharan Africa,Lower middle income


**Bonus Questions:** After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.

1. Adjust your request so that it returns data just for the United States.

In [21]:
US_GDP = gdp_le_region_final[gdp_le_region_final['Country'] == "United States"] 
US_GDP.head(2)

,GDP,Year,Country,Life_Expectancy,Capital,Region,Income Level
13391,74158.716417,2023,United States,78.385366,Washington D.C.,North America,High income
13392,72679.257540,2022,United States,77.434146,Washington D.C.,North America,High income


2. Adjust your request so that it returns data just for the United States for the year 2021.

In [22]:
US_GDP_2021 = US_GDP[US_GDP['Year'] == "2021"] 
US_GDP_2021.head()

,GDP,Year,Country,Life_Expectancy,Capital,Region,Income Level
13393,71307.401728,2021,United States,76.329268,Washington D.C.,North America,High income


3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.